# Notebook for developing code to go into structural_attack class

In [1]:
import sys
import os


#for development use local copy of aisdc in preference to installed version
sys.path.insert(0,os.path.abspath(".."))


In [4]:
import json

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC


from aisdc.attacks.structural_attack import StructuralAttack  # pylint: disable = import-error
from aisdc.attacks.target import Target  # pylint: disable = import-error

In [5]:
# [Researcher] Access a dataset
X, y = load_breast_cancer(return_X_y=True, as_frame=False)

# [Researcher] Split into training and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

In [50]:
# [Researcher] Define the classifier
def get_target(modeltype:str)->Target:
    #these should be handled
    if modeltype =='dt':
        target_model = DecisionTreeClassifier(max_depth=3,min_samples_leaf = 10)
    elif modeltype =='rf':
        target_model = RandomForestClassifier()
    elif modeltype =='xgb':
        target_model = XGBClassifier()
    # should get polite error but not DoF yet    
    elif modeltype == 'svc' :
        target_model = SVC()
    else:
        raise NotImplementedError('model type passed to get_model unknown')
        
    # [Researcher] Train the classifier
    target_model.fit(train_X, train_y)

    # [Researcher] Provide the model and the train and test data to the TRE

    # [TRE] Compute the predictions on the training and test sets
    #train_preds = target_model.predict_proba(train_X)
    #test_preds = target_model.predict_proba(test_X)

    # [TRE / Researcher] Wrap the model and data in a Target object
    target = Target(
        model=target_model)
    target.add_processed_data(train_X, train_y, test_X, test_y)
    
    return target

In [51]:
import importlib
import aisdc.attacks.structural_attack
importlib.reload(aisdc.attacks.structural_attack)
from aisdc.attacks.structural_attack import StructuralAttack

In [52]:
tree_types = ['dt']#,'rf','xgb','svc','other']

for mytype in tree_types:
    print( f'====={mytype} ======')
    target = get_target(mytype)
    myattack = StructuralAttack()
    ret= myattack.attack(target)

    print(ret)

INFO:acro:version: 0.4.2
INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': False}
INFO:acro:automatic suppression: False
INFO:structural_attack:Thresholds for count 10 and DoF 10
INFO:structural_attack:degrees of freedom for this decision tree is 386,DoF risk is 0
INFO:structural_attack:minimum k-anonymity for this tree is 10k-anonymity risk is 0
INFO:structural_attack: for this tree there are 5 problematic regions so class disclosure risk = 1


=====dt ======
None
